### LDA基础

#### Digamma函数

$Ψ(x) = \frac{dlogΓ(x)}{dx}$

<img src="../ml/图片/digamma1.png">

### 文本建模

统计文本建模的目的就是追问这些观察到的语料库中的词序列到底是如何生成的。也就是说我们通过已有的语料来建模。如果把这些我们能看到的文本预料看成是上帝在跑骰子而生成的，那我们只观察到了结果--语料，至于具体是怎么抛的，对我们而言是个黑盒子。

实际上这就需要我们进行文本建模：我们关心两个问题
    
    1、上帝有些啥样的骰子
    2、上帝是如何抛这些骰子的
    
第一个问题就是求骰子各个面的概率，模型参数问题。

第二个问题是指按什么样的规则去抛骰子，从而产生我们看到的词序列。


#### Unigram Model

假设我们的词典有V个词，$v_1,v_2,...v_V$，那么Unigram Model的规则就是下面这样：

<img src="../ml/图片/unigram_model.png">

<img src="../ml/图片/2019-06-11_132710.png">

上面的$P(W)$是没有考虑词的顺序的概率，所以前面的排序系数去掉了。

$P(W)$就是我们的似然概率，所以用极大似然估计法求解，argmax(P(W))

当然在贝叶斯派认为一切皆随机变量，所以上帝手里的骰子也是有多个的，选择哪个骰子，本身也是一个随机变量，符合概率分布。

<img src="../ml/图片/2019-06-11_135601.png">

这样来看，我们模型的参数$p_i$实际上也是服从一个概率分布，这个分布$P(p_i)$可以认为是$p_i$的先验分布

<img src="../ml/图片/2019-06-11_140013.png">

<img src="../ml/图片/2019-06-11_140953.png">
<img src="../ml/图片/2019-06-11_141133.png">

#### Topic Model和PLSA

前的Unigram 模型还是太简单了。有没有更好的模型。

#####  PLSA
人类写文章的时候，往往考虑了主题这一概念，一般一篇文档可以由多个主题（Topic）组成，文档中的每个词都是由一个固定的Topic生成的，所以PLSA的游戏规则为：

<img src="../ml/图片/2019-06-11_142135.png">

实际上，我们看上面的过程中，文档与文档之间的词，同个文档之间的词也是可以交换的，所以它本质上还是一个词袋模型。
<img src="../ml/图片/2019-06-11_143401.png">

先找到似然函数,L = p(W) 。

<img src="../ml/图片/2019-06-11_150932.png">

$n(d_i,w_j)$表示词 $w_j$ 在文档 $d_i$ 中出现的频次

对于给定训练预料，希望公式 (69) 最大化。 $p(z_k|d_i)$和$p(w_j|z_k)$ 是 PLSA 模型需要求解的参数，按照通常的做法是令偏导数 为0，但是参数是以求和的形式出现在对数函数里面，求导后会变得很复杂。n(di)表示第 i 篇文档的词数，所以当预料固定，公式(69)中第一项可以看作常量，所以只要最大化(69)中的第二项即可，如公式(70)所示。

<img src="../ml/图片/2019-06-11_151138.png">

引入$z_k$的概率分布$\sum_{k=1}^{K}Q_k(z_k) = 1$，根据jensen不等式

<img src="../ml/图片/2019-06-11_151547.png">

当 $Q_k(z_k) = p(z_k|w_j,d_i)$时取等号。也就是$Q_k(z_k)$的分布

<img src="../ml/图片/2019-06-11_153814.png">

根据拉格朗日乘子法。对概率$p(z_k|d_i)$和$p(w_j|z_k)$求偏导

<img src="../ml/图片/2019-06-11_154124.png">

1.E-step 随机初始化变量$p(z_k|d_i)$和$p(w_j|z_k)$， 计算隐变量后验概率$Q_k(z_k)$。$Q_k(z_k) = p(z_k|w_j,d_i)$

2.M-step 最大化似然函数，更新变量$p(z_k|d_i)$和$p(w_j|z_k)$

### LDA前提

终于到了LDA，对于 PLSA 模型，贝叶斯学派表示不同意，为什么上帝只有一个 doc-topic 骰子，为什么上帝只有固定 K 个topic-word骰子？模型的参数是固定的，但是这些参数本身应该也是随机变量，服从某种随机变量，所以它肯定要有个先验概率。于是我们把这两个模型参数$p(z_k|d_i)$和$p(w_j|z_k)$分布加上dir先验，因为他们都是multi分布。

<img src="../ml/图片/2019-06-11_154929.png">

所以LDA的玩法就是下面的规则：
<img src="../ml/图片/2019-06-11_155039.png">

实际上这里的LDA有两个物理过程！！！

一定要理解这两个过程。

<img src="../ml/图片/lda概率图.png">


##### 第一个过程。$\alpha->\theta->z_{d,n}->w_{d,n}$

这个过程表示在生成第m 篇文档的时候，先从第一个坛子中抽了一个doc-topic 骰子$\overrightarrow{\theta_m}$, 然后投掷这个骰子生成了文档中第 n 个词的topic编号$z_{m,n}$；

##### 第二个过程。$\eta->\beta_k->w_{d,n}|k=z_{d,n}$

这个过程表示用如下动作生成语料中第m篇文档的第 n个词：在上帝手头的K 个topic-word 骰子$\overrightarrow{φ_k}$中，挑选编号为 $k=z-{m,n}$的那个骰子进行投掷，然后生成 word $w_{m,n}$；


lda实际上还是词袋模型，所以很多过程是可以独立交换的。lda有M+K个独立的共轭结构？这个怎么理解。

从第一个过程可知。$α−>θ−>z_{d,n}$  其实就是一个共轭结构，因为$α−>θ$是 Dir分布，$θ−>z_{d,n}$是Multi分布。所以他们结合是Dir-Multi共轭。等等，这里的$\theta_m$是啥？第m篇文档的主题概率分布，它就是我们第一个过程要求的模型参数。根据前面的经验，参数$\theta_m$的后验 ∝ 似然 * 先验。似然是啥？先验又是啥？？

因为在第 m 篇文档有 Nm 个词，所以需要投掷 Nm 次骰子，为该篇文档中的每个词生成一个主题， 第 n 个词对应的主题为$z_{m,n}$,整篇文档的主题表示为$\overrightarrow{z_m}$。在 Nm 次投掷过程中，每个主题出现的次数为$\overrightarrow{n_m} = (n_{m}^{(1)}, \cdots, n_{m}^{(K)})$，那$\overrightarrow{n_m}$就服从多项式分布，（只生成每个词的主题，并未由主题产生具体的词）。

所以参数$\theta$的先验是不是就是$Dir(\overrightarrow{\theta_m}|\overrightarrow{\alpha})$对吧，似然呢？？？ 是不是就是第m篇文档中，每个主题分量$z_m$的个数，它是一个multi多选分布，文档中选多少个主题嘛，就是上面描述的$Multi(\overrightarrow{n_m}|\overrightarrow{\theta_m},N_m)$。

所以参数的后验估计$\overrightarrow{\theta_m}$:

$Dir(\overrightarrow{\theta}_m| \overrightarrow{n}_m + \overrightarrow{\alpha})$

因此我们计算出第m篇文档的主题概率分布：
<img src="../ml/图片/2019-06-12_154947.png">

由于语料中 M篇文档的 topics 生成过程相互独立，所以我们得到 M 个相互独立的 Dirichlet-Multinomial 共轭结构，从而我们可以得到整个语料中 topics 生成概率。

<img src="../ml/图片/2019-06-12_155210.png">

至此，我们第一个过程就梳理完毕。这里总结一下，最重要的是理解，

    1、模型的参数是啥？
    2、我们的观察值$\overrightarrow{n_m}$实际上是不能观察到的。
    3、这M个dir-multi共轭过程是相互独立的

整理一下，生成每个词，都要抛两次骰子，一次选择主题，接着选择主题下的词。如果语料有N个词，那么就有2N次抛骰子，但实际上有一些抛骰子的顺序是可以交换的，我们可以等价的调整2N次抛骰子的次序：前N次只抛doc-topic骰子得到语料中所有词的 topics,然后基于得到的每个词的 topic 编号，后N次只抛topic-word骰子生成 N 个word。于是上帝在玩 LDA 游戏的时候，可以等价的按照如下过程进行：
<img src="../ml/图片/game-lda-2.jpg">

以上游戏是先生成了语料中所有词的 topic, 然后对每个词在给定 topic 的条件下生成 word。考虑到在语料中所有词的 topic 已经生成的条件下，任何两个 word 的生成动作都是可交换的。于是我们把语料中的词进行交换，把具有相同 topic 的词放在一起。

在看第二个过程：
$\overrightarrow{\beta} \rightarrow \overrightarrow{\varphi}_k \rightarrow w_{m,n} | k=z_{m,n}$。在 $k=z_{m,n}$ 的限制下，语料中任何两个由 topic k 生成的词都是可交换的，即便他们不再同一个文档中，所以我们此处不再考虑文档的概念，转而考虑由同一个 topic 生成的词。

<img src="../ml/图片/lda-graph-model-300x255.jpg">

$\overrightarrow{\beta} \rightarrow \overrightarrow{\varphi}_k$是Dir分布，$\overrightarrow{\varphi}_k \rightarrow \overrightarrow{w}_{(k)}$是Multi分布。

主题z有 K 种不同的取值（因为我们假设语料有 K 个主题），我们现在为第 k 个主题生成对应的词，那么需要选择编号为 k 的topic-word骰子，该骰子有 V 个面，每个面表示一个词，那么在一次投掷骰子过程中，每个词的概率为 $\overrightarrow{n}_k = (n_{k}^{(1)}, \cdots, n_{k}^{(V)})$，其中$n_{k}^{(t)}$表示第k 个topic 产生的词中 word t的个数，所以要抛$n_{k}^{(t)}$次骰子。

参数是啥？ $\overrightarrow{\varphi}_k$。先验是：
$Dir(\overrightarrow{\varphi}_k|\overrightarrow{\beta})$ 。对吧。第k个主题下的词的概率分布。

似然是啥？第k个主题下，出现的词的个数呗。$Multi(\overrightarrow{n}_k|\overrightarrow{\varphi}_k,\overrightarrow{\beta})$

后验是啥？？
<img src="../ml/图片/2019-06-12_173548.png">

第 k 个主题的词概率分布：
<img src="../ml/图片/2019-06-12_173747.png">

而整个语料中的 K 个主题是相互独立的，所以可以得到语料中词的概率为：
<img src="../ml/图片/2019-06-12_174200.png">

我们把过程1和过程1结合起来。

<img src="../ml/图片/2019-06-12_174450.png">

### Gibbs采样求解

为啥要求p(z|w)这个条件概率

<img src="../ml/图片/2019-06-13_102758.png">

有了联合分布(所以说lda是生成模型。) $p(\overrightarrow{\mathbf{w}},\overrightarrow{\mathbf{z}})$, 万能的 MCMC 算法就可以发挥作用了。不过别急，我们看，上面的公式$***$,$\overrightarrow{n_k}$和$\overrightarrow{n_m}$都是无法直接观测到，主要是由于我们的$\overrightarrow{z}$是隐变量，它表示语料中每个词对应的主题，只要我们知道了$\overrightarrow{z}$，那$\overrightarrow{n_k}$和$\overrightarrow{n_m}$都能求出来。所以我们实际上采样的是是条件分布$p(\overrightarrow{z}|\overrightarrow{w})$，把$\overrightarrow{z}$给它采样出来。

<img src="../ml/图片/2019-06-12_180241.png">

$\begin{align*}p\left( {{z_i} = k|{{\vec z}_{ - i}},\vec w} \right) &= \frac{{p\left( {\vec w,\vec z} \right)}}{{p\left( {\vec w,{{\vec z}_{ - i}}} \right)}}\\
 &= \frac{{p\left( {\vec w|\vec z} \right)}}{{p\left( {{{\vec w}_{ - i}}|{{\vec z}_{ - i}}} \right)p\left( {{w_i}} \right)}} \cdot \frac{{p\left( {\vec z} \right)}}{{p\left( {{{\vec z}_{ - i}}} \right)}}\\
 &\propto \frac{{\Delta \left( {{{\vec n}_k} + \vec \beta } \right)}}{{\Delta \left( {{{\vec n}_{k, - i}} + \vec \beta } \right)}} \cdot \frac{{\Delta \left( {{{\vec n}_m} + \vec \alpha } \right)}}{{\Delta \left( {{{\vec n}_{m, - i}} + \vec \alpha } \right)}}\\
 &= \frac{{n_{k, - i}^{\left( t \right)} + {\beta ^{\left( t \right)}}}}{{\sum\nolimits_{v = 1}^V {\left( {n_{k, - i}^{\left( v \right)} + {\beta ^{\left( v \right)}}} \right)} }} \cdot \frac{{n_{m, - i}^{\left( k \right)} + {\alpha ^{\left( k \right)}}}}{{\sum\nolimits_{j = 1}^K {\left( {n_{m, - i}^{\left( j \right)} + {\alpha ^{\left( j \right)}}} \right)} }} \tag{84}
\end{align*}$

#### 上面的推导过程堪称MCMC经典，神奇的是，我们的gamma函数竟然消失了。。。

当然利用了贝叶斯公式:

$p(a,b,c)=p(a|b,c)* p(b|c) * p(c)$

$p(\overrightarrow{z}_{-i}|w_i) = p(\overrightarrow{z}_{-i})    其中\overrightarrow{z}_{-i} 与 w_i 独立$  

同理：
$p(\overrightarrow{w}_{-i}|\overrightarrow{z}_{-i}, w_i) = p(\overrightarrow{w}_{-i}|\overrightarrow{z}_{-i})$

$\frac{\Gamma(n)}{\Gamma(n+1)} = \frac{(n-1)!}{n!} = \frac{(n-1)*(n-2)...(1)}{(n)*(n-1)...(1)} = \frac{1}{n}$

$\Delta(\vec{\alpha}) = \frac{\prod\limits_{k = 1}^K\Gamma(\alpha_k)}{\Gamma(\sum\limits_{k = 1}^K(\alpha_k)}$

由于zi=k,wi=t 只涉及到第 m 篇文档和第k个 topic，所以上式的条件概率计算中, 实际上也只会涉及到如下两个Dirichlet-Multinomial 共轭结构，而其他的 M+K−2 个 Dirichlet-Multinomial 共轭结构和zi=k,wi=t是独立的，可以约掉，这样公式就能简化了。
$\begin{align*}p\left( {\vec w|\vec z} \right) &= \Delta \left( {\vec \beta  + {{\vec n}_k}} \right)\prod\limits_{z = 1,z \ne k}^K {\frac{{\Delta \left( {\vec \beta  + {{\vec n}_z}} \right)}}{{\Delta \left( {\vec \beta } \right)}}} \\
p\left( {{{\vec w}_{ - i}}|{{\vec z}_{ - i}}} \right) &= \Delta \left( {\vec \beta  + {{\vec n}_{k, - i}}} \right)\prod\limits_{z = 1,z \ne k}^K {\frac{{\Delta \left( {\vec \beta  + {{\vec n}_z}} \right)}}{{\Delta \left( {\vec \beta } \right)}}} \\
p\left( {\vec z} \right) &= \Delta \left( {\vec \alpha  + {{\vec n}_m}} \right)\prod\limits_{i = 1,i \ne m}^M {\frac{{\Delta \left( {\vec \alpha  + {{\vec n}_i}} \right)}}{{\Delta \left( {\vec \alpha } \right)}}} \\
p\left( {{{\vec z}_{ - i}}} \right) &= \Delta \left( {\vec \alpha  + {{\vec n}_{m, - i}}} \right)\prod\limits_{i = 1,i \ne m}^M {\frac{{\Delta \left( {\vec \alpha  + {{\vec n}_i}} \right)}}{{\Delta \left( {\vec \alpha } \right)}}} 
\tag{85} \end{align*}$



另一种是解法是

$\begin{align*} 
p(\overrightarrow{\theta}_m|\overrightarrow{\mathbf{z}}_{\neg i}, \overrightarrow{\mathbf{w}}_{\neg i}) 
&= Dir(\overrightarrow{\theta}_m| \overrightarrow{n}_{m,\neg i} + \overrightarrow{\alpha}) \\ 
p(\overrightarrow{\varphi}_k|\overrightarrow{\mathbf{z}}_{\neg i}, \overrightarrow{\mathbf{w}}_{\neg i}) 
&= Dir( \overrightarrow{\varphi}_k| \overrightarrow{n}_{k，\neg i} + \overrightarrow{\beta}) 
\end{align*}$

$\begin{align*}p\left( {{z_i} = k|{{\vec z}_{ - i}},\vec w} \right) &\propto p\left( {{z_i} = k,{w_i} = t|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right)\\
 &= \int {p\left( {{z_i} = k,{w_i} = t,{{\vec \vartheta }_m},{{\vec \varphi }_k}|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right)} d{{\vec \vartheta }_m}d{{\vec \varphi }_k}\\
 &= \int {p\left( {{z_i} = k,{{\vec \vartheta }_m}|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right) \cdot } p\left( {{w_i} = t,{{\vec \varphi }_k}|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right)d{{\vec \vartheta }_m}d{{\vec \varphi }_k}\\
 &= \int {p\left( {{z_i} = k|{{\vec \vartheta }_m}} \right)p\left( {{{\vec \vartheta }_m}|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right) \cdot } p\left( {{w_i} = t|{{\vec \varphi }_k}} \right)p\left( {{{\vec \varphi }_k}|{{\vec z}_{ - i}},{{\vec w}_{ - i}}} \right)d{{\vec \vartheta }_m}d{{\vec \varphi }_k}\\
 &= \int {p\left( {{z_i} = k|{{\vec \vartheta }_m}} \right)Dir\left( {{{\vec \vartheta }_m}|{{\vec n}_{m, - i}} + \vec \alpha } \right) \cdot } p\left( {{w_i} = t|{{\vec \varphi }_k}} \right)Dir\left( {{{\vec \varphi }_k}|{{\vec n}_{k, - i}} + \vec \beta } \right)d{{\vec \vartheta }_m}d{{\vec \varphi }_k}\\
 &= \int {\vartheta _m^{\left( k \right)}Dir\left( {{{\vec \vartheta }_m}|{{\vec n}_{m, - i}} + \vec \alpha } \right)d{{\vec \vartheta }_m}}  \cdot \int {\varphi _k^{\left( t \right)}Dir\left( {{{\vec \varphi }_k}|{{\vec n}_{k, - i}} + \vec \beta } \right)d{{\vec \varphi }_k}} \\
 &= E\left( {\vartheta _m^{\left( k \right)}} \right) \cdot E\left( {\varphi _k^{\left( t \right)}} \right)\\
 &= \frac{{n_{m, - i}^{\left( k \right)} + {\alpha ^{\left( k \right)}}}}{{\sum\nolimits_{j = 1}^K {\left( {n_{m, - i}^{\left( j \right)} + {\alpha ^{\left( j \right)}}} \right)} }} \cdot \frac{{n_{k, - i}^{\left( t \right)} + {\beta ^{\left( t \right)}}}}{{\sum\nolimits_{v = 1}^V {\left( {n_{k, - i}^{\left( v \right)} + {\beta ^{\left( v \right)}}} \right)} }}\end{align*}$

<img src="../ml/图片/2019-06-13_103711.png">

等等，说了一大堆，lda模型的参数是啥？ 

$\overrightarrow{\varphi}_1, \cdots, \overrightarrow{\varphi}_K$

$\overrightarrow{\theta}_1, \cdots, \overrightarrow{\theta}_M$

有了模型的参数，对于新来的一篇文档docnew，我们能够计算这篇文档的 topic 分布$\overrightarrow{\theta}_{new}$。

有了 Gibbs Sampling 公式， 我们就可以基于语料训练 LDA 模型，并应用训练得到的模型对新的文档进行 topic 语义分析。训练的过程就是获取语料中的 (z,w) 的样本，而模型中的所有的参数都可以基于最终采样得到的样本进行估计。训练的流程很简单:

<img src="../ml/图片/lda-training.jpg">

<img src="../ml/图片/2019-06-13_104944.png">

### 变分法求解

<img src = "../ml/图片/126459846.jpg"> 

<img src = "../ml/图片/1903181305.jpg"> 

<img src = "../ml/图片/783553958.jpg"> 

<img src = "../ml/图片/837423007.jpg"> 

<img src = "../ml/图片/327863916.jpg"> 